In [1]:
import torch
from torch import Tensor
from torch.utils import data as torch_data
from torch.amp import autocast, GradScaler

from models import UNet
from datasets import PreprocessedOpenFWI

In [2]:
# model
model = UNet(
    in_channels=5,
    out_channels=1,
    start_features=32,
    depth=4
).cuda()
# data
train_dataset = PreprocessedOpenFWI(train=True, norm_output=True, nb_files_to_load=1)
train_loader = torch_data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_dataset = PreprocessedOpenFWI(train=False, norm_output=True)
test_loader = torch_data.DataLoader(train_dataset, batch_size=128, shuffle=True)
# Training
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
scaler = GradScaler(device="cuda")

100%|██████████| 20.6G/20.6G [17:04<00:00, 21.6MB/s]  

Extracting files...


Output()

Output()

Output()

Output()

In [8]:
step = 0
for epoch in range(20):
    total_loss = 0
    total_accuracy = 0
    # Count nb samples instead of accessing len(data_loader.dataset)
    # in case the data lauder augments the number of samples
    nb_batches = 0
    nb_samples = 0
    for batch_x, batch_y in train_loader:
        batch_x = batch_x.cuda()
        batch_y = batch_y.cuda()
        nb_batches += 1
        nb_samples += len(batch_x)
        model.train()
        optimizer.zero_grad()
        # print(batch_x.dtype)
        with autocast(device_type="cuda"):
            batch_y_pred = model(batch_x)

        loss_value = (batch_y_pred - batch_y).abs().mean()
        scaler.scale(loss_value).backward()
        # scaler.unscale_(optimizer)
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss_value.item()
        step += 1
    print("loss:", total_loss / len(train_loader))


loss: 0.200897216796875
loss: 0.1962432861328125
loss: 0.199462890625
loss: 0.1912689208984375
loss: 0.1862640380859375
loss: 0.1873016357421875
loss: 0.171142578125
loss: 0.1835784912109375
loss: 0.1676177978515625
loss: 0.1704864501953125
loss: 0.1624755859375
loss: 0.164642333984375
loss: 0.1561737060546875
loss: 0.1641845703125
loss: 0.1605072021484375
loss: 0.1559600830078125
loss: 0.148834228515625
loss: 0.1575469970703125
loss: 0.1555328369140625
loss: 0.15814208984375
